In [1]:
# Install the 'proxop' library, which provides proximal operators for optimization algorithms.
# This command should be run in a Jupyter Notebook or Google Colab environment.
# The exclamation mark (!) is used to run shell commands from within Python notebooks.
!pip install proxop

In [2]:
# Import necessary libraries
import numpy as np # For numerical operations
import pandas as pd  # For data manipulation using DataFrames
from sklearn import datasets # For loading standard datasets from scikit-learn
import time  # For measuring execution time

# Load the diabetes dataset
diabetes = datasets.load_diabetes()

# Print dataset description
print(diabetes.DESCR)

# Extract the feature matrix (inputs) from the dataset
X = diabetes.data # Shape: (442, 10)
# Extract the target vector
y= diabetes.target


.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [3]:
# Replace NaNs in X with column means
col_means = np.nanmean(X, axis=0)
inds = np.where(np.isnan(X))
X[inds] = np.take(col_means, inds[1])

# Replace NaNs in y with mean
y = np.nan_to_num(y, nan=np.nanmean(y))

# Standardize features (recommended for gradient descent)
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

# Add intercept term (bias)
X = np.c_[np.ones(X.shape[0]), X]

In [4]:
# Compute Lipschitz constant L = ||X||_2^2 / m
#Perform Singular Value Decomposition (SVD) on X
U, S, Vt = np.linalg.svd(X, full_matrices=False)
# The spectral norm is the largest singular value of X
spectral_norm = S[0]
# Compute Lipschitz constant: L = (spectral_norm)^2 / number of samples
L = (spectral_norm ** 2) / X.shape[0]
# Compute the step size
eta = 1 / L  # Step size

In [5]:
print(f"Lipschitz constant: {L}")
print(f"Step size: {1/L}")


Lipschitz constant: 4.024210750152778
Step size: 0.24849593177048077


In [6]:

# Define the mean squared error (MSE) loss function
def loss(beta, X, y):
    """
    Computes the Mean Squared Error loss.

    Parameters:
    - beta: Coefficient vector (numpy array)
    - X: Feature matrix (numpy array)
    - y: Target vector

    Returns:
    - MSE loss value (float)
    """
    return np.mean((X @ beta - y) ** 2)


# Backtracking line search
def backtracking_line_search(X, y, beta, grad, L, t_init=1, alpha=0.9, c=1e-4):
    """
    Performs backtracking line search to find a suitable step size.

    Parameters:
    - X, y: Feature matrix and target vector
    - beta: Current parameter estimate
    - grad: Gradient of loss at beta
    - L: Lipschitz constant
    - t_init: Initial step size
    - alpha: Step size shrinkage factor (0 < alpha < 1)
    - c: Armijo condition constant (typically small, e.g. 1e-4)

    Returns:
    - t: Chosen step size
    """
    t_min = min(t_init, alpha / float(L)) # Prevent step size from shrinking too small
    t = t_min
    direction = -grad # Descent direction
    loss_old = loss(beta, X, y)
    # Line search loop
    while True:
        beta_new = beta + t * direction
        loss_new = loss(beta_new, X, y)
        # Armijo condition
        if loss_new <= loss_old + c * t * np.dot(grad, direction):
            break
        t *= alpha
        if t < t_min:
            t = t_min
            break
    return t

# Gradient Descent with Backtracking Line Search
def gradient_descent_backtracking_L(X, y, L, epsilon=1e-6):
    """
    Performs gradient descent using backtracking line search and a Lipschitz estimate.

    Parameters:
    - X, y: Feature matrix and target vector
    - L: Lipschitz constant
    - epsilon: Convergence tolerance

    Returns:
    - beta: Optimal parameter vector
    """
    n, d = X.shape # Initialize parameters
    beta = np.ones(d)
    t_init=1   # Initial step size
    i=0 # Iteration counter
    while(True):
        i+=1
        grad = (2 / n) * X.T @ (X @ beta - y) # Compute gradient
        beta_prev = beta.copy() # Store previous parameters
        # Backtracking line search to get step size
        eta = backtracking_line_search(X, y, beta, grad, L,t_init)
        t_init=eta  # Update initial step size for next iteration
        # Gradient descent step
        beta = beta_prev - eta * grad
        # Check relative change in loss for convergence
        if abs(loss(beta,X,y) - loss(beta_prev,X,y))/max(1,abs(loss(beta_prev,X,y))) < epsilon:
          print(f"Converged in {i} iterations.")
          break
        print(f"Iter {i+1}: step_size = {eta:.4e}")
    return beta
# Run optimization and measure computation time
start_time = time.time()
beta_opt = gradient_descent_backtracking_L(X, y, L)
end_time = time.time()
print(f"Obj: {loss(beta_opt,X,y)}")  # Final objective value
print(f"Running time= {end_time-start_time}") # Execution time

Iter 2: step_size = 2.2365e-01
Iter 3: step_size = 2.2365e-01
Iter 4: step_size = 2.2365e-01
Iter 5: step_size = 2.2365e-01
Iter 6: step_size = 2.2365e-01
Iter 7: step_size = 2.2365e-01
Iter 8: step_size = 2.2365e-01
Iter 9: step_size = 2.2365e-01
Iter 10: step_size = 2.2365e-01
Iter 11: step_size = 2.2365e-01
Iter 12: step_size = 2.2365e-01
Iter 13: step_size = 2.2365e-01
Iter 14: step_size = 2.2365e-01
Iter 15: step_size = 2.2365e-01
Iter 16: step_size = 2.2365e-01
Iter 17: step_size = 2.2365e-01
Iter 18: step_size = 2.2365e-01
Iter 19: step_size = 2.2365e-01
Iter 20: step_size = 2.2365e-01
Iter 21: step_size = 2.2365e-01
Iter 22: step_size = 2.2365e-01
Iter 23: step_size = 2.2365e-01
Iter 24: step_size = 2.2365e-01
Iter 25: step_size = 2.2365e-01
Iter 26: step_size = 2.2365e-01
Iter 27: step_size = 2.2365e-01
Iter 28: step_size = 2.2365e-01
Iter 29: step_size = 2.2365e-01
Iter 30: step_size = 2.2365e-01
Iter 31: step_size = 2.2365e-01
Iter 32: step_size = 2.2365e-01
Iter 33: step_si

In [7]:
# Gradient descent Fixed step size
def gradient_descent_L(X, y, L, epsilon=1e-6):
    n, d = X.shape
    beta = np.ones(d)
    step_size=1/L
    i=0
    while(True):
        i+=1
        grad = (2 / n) * X.T @ (X @ beta - y)
        beta_prev = beta.copy()
        beta = beta_prev - step_size * grad
        if abs(loss(beta,X,y) - loss(beta_prev,X,y))/max(1,abs(loss(beta_prev,X,y))) < epsilon:
          print(f"Converged in {i} iterations.")
          break
        print(f"Iter {i+1}: step_size = {step_size:.4e}")
    return beta
start_time = time.time()
beta_opt = gradient_descent_backtracking_L(X, y, L)
end_time = time.time()
print(f"Obj: {loss(beta_opt,X,y)}")
print(f"Running time= {end_time-start_time}")

Iter 2: step_size = 2.2365e-01
Iter 3: step_size = 2.2365e-01
Iter 4: step_size = 2.2365e-01
Iter 5: step_size = 2.2365e-01
Iter 6: step_size = 2.2365e-01
Iter 7: step_size = 2.2365e-01
Iter 8: step_size = 2.2365e-01
Iter 9: step_size = 2.2365e-01
Iter 10: step_size = 2.2365e-01
Iter 11: step_size = 2.2365e-01
Iter 12: step_size = 2.2365e-01
Iter 13: step_size = 2.2365e-01
Iter 14: step_size = 2.2365e-01
Iter 15: step_size = 2.2365e-01
Iter 16: step_size = 2.2365e-01
Iter 17: step_size = 2.2365e-01
Iter 18: step_size = 2.2365e-01
Iter 19: step_size = 2.2365e-01
Iter 20: step_size = 2.2365e-01
Iter 21: step_size = 2.2365e-01
Iter 22: step_size = 2.2365e-01
Iter 23: step_size = 2.2365e-01
Iter 24: step_size = 2.2365e-01
Iter 25: step_size = 2.2365e-01
Iter 26: step_size = 2.2365e-01
Iter 27: step_size = 2.2365e-01
Iter 28: step_size = 2.2365e-01
Iter 29: step_size = 2.2365e-01
Iter 30: step_size = 2.2365e-01
Iter 31: step_size = 2.2365e-01
Iter 32: step_size = 2.2365e-01
Iter 33: step_si

In [8]:
# Sample data
n, d = X.shape
lambda_reg = 0.1
# Parameters
L = np.linalg.norm(X, ord=2) ** 2 / n  # Lipschitz constant
def lasso_objective(beta,X,y):
    residual = X @ beta - y
    return (1 / (2 * n)) * np.sum(np.dot(residual,residual )) + lambda_reg * np.linalg.norm(beta, 1)


In [9]:
#Fixed step size
from proxop import L1Norm
def ista(X, y, L, lambda_reg=0.1, max_iter=1000, epsilon=1e-6):
  """
    Iterative Soft-Thresholding Algorithm (ISTA) to solve:
        min_beta (1/2n)||Xβ - y||² + λ||β||₁

    Parameters:
    - X: Feature matrix (standardized)
    - y: Target vector
    - lambda_: Regularization parameter controlling sparsity
    - step_size: Learning rate (must be <= 1 / Lipschitz constant of ∇f)
    - max_iter: Maximum number of iterations
    - epsilon: Convergence tolerance for stopping criterion

    Returns:
    - beta: Estimated coefficient vector
    """
  # Initialize parameters
  step_size = 1 / L
  beta = np.ones(d)

  # Define the proximal operator for L1 regularization
  prox_g = L1Norm(lambda_reg)

  # Proximal Gradient Descent loop
  for i in range(max_iter):
      gradient = (1 / n) * X.T @ (X @ beta - y)  # Gradient of MSE
      norm = np.linalg.norm(gradient)
      beta_prev = beta.copy()
      beta = prox_g.prox(beta - step_size * gradient, step_size) # Proximal update
      print(f"Iter {i}: Obj = {lasso_objective(beta,X,y):.6f}")

      # Check for convergence
      if abs(lasso_objective(beta,X,y) - lasso_objective(beta_prev,X,y))/max(1,abs(lasso_objective(beta_prev,X,y))) < epsilon:
          print(f"Converged in {i} iterations")
          break

  # Final beta
  return beta


start_time = time.time()
beta_opt= ista(X,y,L)
end_time= time.time()
print(f"Obj = {lasso_objective(beta_opt,X,y):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8270.672880
Iter 1: Obj = 5329.045639
Iter 2: Obj = 3668.846642
Iter 3: Obj = 2728.546706
Iter 4: Obj = 2193.469277
Iter 5: Obj = 1888.039455
Iter 6: Obj = 1713.143942
Iter 7: Obj = 1612.539628
Iter 8: Obj = 1554.308485
Iter 9: Obj = 1520.346478
Iter 10: Obj = 1500.356690
Iter 11: Obj = 1488.458228
Iter 12: Obj = 1481.279747
Iter 13: Obj = 1476.879568
Iter 14: Obj = 1474.132868
Iter 15: Obj = 1472.383253
Iter 16: Obj = 1471.244208
Iter 17: Obj = 1470.485622
Iter 18: Obj = 1469.968685
Iter 19: Obj = 1469.608381
Iter 20: Obj = 1469.351728
Iter 21: Obj = 1469.165084
Iter 22: Obj = 1469.026657
Iter 23: Obj = 1468.922042
Iter 24: Obj = 1468.841529
Iter 25: Obj = 1468.778450
Iter 26: Obj = 1468.728147
Iter 27: Obj = 1468.687321
Iter 28: Obj = 1468.653601
Iter 29: Obj = 1468.625268
Iter 30: Obj = 1468.601061
Iter 31: Obj = 1468.580053
Iter 32: Obj = 1468.561552
Iter 33: Obj = 1468.545042
Iter 34: Obj = 1468.530134
Iter 35: Obj = 1468.516536
Iter 36: Obj = 1468.504023
Iter 37: Ob

In [10]:
#Adaptive step size
from proxop import L1Norm
def ista(X, y, L, lambda_reg=0.1, max_iter=1000, epsilon=1e-6):
  """
    Iterative Soft-Thresholding Algorithm (ISTA) to solve:
        min_beta (1/2n)||Xβ - y||² + λ||β||₁

    Parameters:
    - X: Feature matrix (standardized)
    - y: Target vector
    - lambda_: Regularization parameter controlling sparsity
    - step_size: Learning rate (must be <= 1 / Lipschitz constant of ∇f)
    - max_iter: Maximum number of iterations
    - epsilon: Convergence tolerance for stopping criterion

    Returns:
    - beta: Estimated coefficient vector
    """
  # Initialize parameters
  t_init=1
  beta = np.ones(d)

  # Define the proximal operator for L1 regularization
  prox_g = L1Norm(lambda_reg)

  # Proximal Gradient Descent loop
  for i in range(max_iter):
      gradient = (1 / n) * X.T @ (X @ beta - y)  # Gradient of MSE
      norm = np.linalg.norm(gradient)
      beta_prev = beta.copy()
      eta = backtracking_line_search(X, y, beta, gradient, L,t_init)
      t_init=eta
      beta = prox_g.prox(beta - eta * gradient, eta)
      print(f"Iter {i}: Obj = {lasso_objective(beta,X,y):.6f}")

      # Check for convergence
      if abs(lasso_objective(beta,X,y) - lasso_objective(beta_prev,X,y))/max(1,abs(lasso_objective(beta_prev,X,y))) < epsilon:
          print(f"Converged in {i} iterations")
          break

  # Final beta
  return beta


start_time = time.time()
beta_opt= ista(X,y,L)
end_time= time.time()
print(f"Obj = {lasso_objective(beta_opt,X,y):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8737.801791
Iter 1: Obj = 5860.787710
Iter 2: Obj = 4134.844259
Iter 3: Obj = 3093.047368
Iter 4: Obj = 2461.641037
Iter 5: Obj = 2078.066948
Iter 6: Obj = 1844.483259
Iter 7: Obj = 1701.887785
Iter 8: Obj = 1614.492117
Iter 9: Obj = 1560.660695
Iter 10: Obj = 1527.311027
Iter 11: Obj = 1506.507208
Iter 12: Obj = 1493.421883
Iter 13: Obj = 1485.110343
Iter 14: Obj = 1479.770305
Iter 15: Obj = 1476.294202
Iter 16: Obj = 1473.998000
Iter 17: Obj = 1472.456680
Iter 18: Obj = 1471.404230
Iter 19: Obj = 1470.672710
Iter 20: Obj = 1470.155018
Iter 21: Obj = 1469.782050
Iter 22: Obj = 1469.508636
Iter 23: Obj = 1469.304825
Iter 24: Obj = 1469.150453
Iter 25: Obj = 1469.031728
Iter 26: Obj = 1468.939067
Iter 27: Obj = 1468.865711
Iter 28: Obj = 1468.806818
Iter 29: Obj = 1468.758877
Iter 30: Obj = 1468.719309
Iter 31: Obj = 1468.686199
Iter 32: Obj = 1468.658115
Iter 33: Obj = 1468.633973
Iter 34: Obj = 1468.612950
Iter 35: Obj = 1468.594417
Iter 36: Obj = 1468.577890
Iter 37: Ob

In [11]:
#Fixed step size
from proxop import L1Norm



# Parameters
step_size = 1 / L
max_iter = 1000
epsilon = 1e-6
def fista(X, y, L, lambda_reg=0.1, max_iter=1000, epsilon=1e-6):
    """
    Fast Iterative Soft-Thresholding Algorithm (FISTA) for LASSO:
        Solves: min_beta (1/2n)||Xβ - y||² + λ||β||₁

    Parameters:
    - X (ndarray): Feature matrix (n_samples, n_features), should be standardized
    - y (ndarray): Target vector (n_samples,)
    - L (float): Lipschitz constant of the gradient ∇f = (1/n) * Xᵀ(Xβ - y)
    - lambda_reg (float): L1 regularization parameter controlling sparsity
    - max_iter (int): Maximum number of iterations
    - epsilon (float): Convergence tolerance on relative objective change

    Returns:
    - beta (ndarray): Optimized coefficient vector
      """
    # Initialize parameters
    d = X.shape[1]
    n = X.shape[0]
    beta = np.ones(d)          # Current estimate
    theta = beta.copy()         # Extrapolated point
    t = 1                       # FISTA momentum parameter


    # Define the proximal operator for L1 regularization
    prox_g = L1Norm(lambda_reg)

    # FISTA loop
    for i in range(max_iter):
        gradient = (1 / n) * X.T @ (X @ theta - y)  # Gradient at theta
        beta_next = prox_g.prox(theta - step_size * gradient, step_size)

        # Momentum update
        t_next = (1 + np.sqrt(1 + 4 * t**2)) / 2
        theta = beta_next + ((t - 1) / t_next) * (beta_next - beta)
        print(f"Iter {i}: Obj = {lasso_objective(beta_next,X,y):.6f}")

        # Check for convergence
        if abs(lasso_objective(beta_next,X,y) - lasso_objective(beta,X,y))/max(1,abs(lasso_objective(beta,X,y))) < epsilon:
            print(f"Converged in {i} iterations")
            break

        # Update variables
        beta = beta_next
        t = t_next

    # Final beta
    return beta
start_time = time.time()
beta_opt= fista(X,y,L)
end_time = time.time()
print(f"Obj = {lasso_objective(beta_opt,X,y):.6f}")
print(f"Running time= {end_time-start_time}")

Iter 0: Obj = 8270.672880
Iter 1: Obj = 5329.045639
Iter 2: Obj = 3285.495028
Iter 3: Obj = 2140.963439
Iter 4: Obj = 1639.925580
Iter 5: Obj = 1487.547522
Iter 6: Obj = 1469.805172
Iter 7: Obj = 1478.144743
Iter 8: Obj = 1479.261034
Iter 9: Obj = 1474.009908
Iter 10: Obj = 1469.425176
Iter 11: Obj = 1468.302112
Iter 12: Obj = 1469.409008
Iter 13: Obj = 1470.641984
Iter 14: Obj = 1470.949273
Iter 15: Obj = 1470.431174
Iter 16: Obj = 1469.630785
Iter 17: Obj = 1468.959247
Iter 18: Obj = 1468.545574
Iter 19: Obj = 1468.348394
Iter 20: Obj = 1468.094764
Iter 21: Obj = 1468.025379
Iter 22: Obj = 1467.938603
Iter 23: Obj = 1467.710130
Iter 24: Obj = 1467.484457
Iter 25: Obj = 1467.388895
Iter 26: Obj = 1467.389260
Converged in 26 iterations
Obj = 1467.388895
Running time= 0.00891423225402832


In [12]:
#Adaptive step size
from proxop import L1Norm



# Parameters
L = np.linalg.norm(X, ord=2) ** 2 / n  # Lipschitz constant

max_iter = 1000
epsilon = 1e-6
def fista(X, y, L, lambda_reg=0.1, max_iter=1000, epsilon=1e-6):
  # Initialize parameters
  t_init= 1
  beta = np.ones(d)          # Current estimate
  theta = beta.copy()         # Extrapolated point
  t = 1                       # FISTA momentum parameter

  # Define the proximal operator for L1 regularization
  prox_g = L1Norm(lambda_reg)

  # FISTA loop
  for i in range(max_iter):
      gradient = (1 / n) * X.T @ (X @ theta - y)  # Gradient at theta
      eta = backtracking_line_search(X, y, beta, gradient, L,t_init)
      t_init=eta
      beta_next = prox_g.prox(theta - eta * gradient, eta)

      # Momentum update
      t_next = (1 + np.sqrt(1 + 4 * t**2)) / 2
      theta = beta_next + ((t - 1) / t_next) * (beta_next - beta)
      print(f"Iter {i}: Obj = {lasso_objective(beta_next,X,y):.6f}")

      # Check for convergence
      if abs(lasso_objective(beta_next,X,y) - lasso_objective(beta,X,y))/max(1,abs(lasso_objective(beta,X,y))) < epsilon:
          print(f"Converged in {i} iterations")
          break

      # Update variables
      beta = beta_next
      t = t_next

  # Final beta
  return beta
start_time = time.time()
beta_opt= fista(X,y,L)
end_time = time.time()
print(f"Obj = {lasso_objective(beta_opt,X,y):.6f}")
print(f"Running time= {end_time-start_time}")

Iter 0: Obj = 8737.801791
Iter 1: Obj = 5860.787710
Iter 2: Obj = 3725.523632
Iter 3: Obj = 2416.770641
Iter 4: Obj = 1766.105723
Iter 5: Obj = 1522.454196
Iter 6: Obj = 1469.791995
Iter 7: Obj = 1475.282322
Iter 8: Obj = 1482.163881
Iter 9: Obj = 1479.432013
Iter 10: Obj = 1472.991767
Iter 11: Obj = 1468.827124
Iter 12: Obj = 1468.289098
Iter 13: Obj = 1469.671766
Iter 14: Obj = 1470.952202
Iter 15: Obj = 1471.215358
Iter 16: Obj = 1470.613743
Iter 17: Obj = 1469.708063
Iter 18: Obj = 1468.934030
Iter 19: Obj = 1468.443028
Iter 20: Obj = 1468.196892
Iter 21: Obj = 1467.963931
Iter 22: Obj = 1467.852637
Iter 23: Obj = 1467.665893
Iter 24: Obj = 1467.492094
Iter 25: Obj = 1467.371741
Iter 26: Obj = 1467.349343
Iter 27: Obj = 1467.408359
Iter 28: Obj = 1467.503156
Iter 29: Obj = 1467.591300
Iter 30: Obj = 1467.659696
Iter 31: Obj = 1467.707828
Iter 32: Obj = 1467.737767
Iter 33: Obj = 1467.751998
Iter 34: Obj = 1467.753894
Iter 35: Obj = 1467.747831
Iter 36: Obj = 1467.738455
Iter 37: Ob

In [13]:


def ridge_objective(beta, X, y, lambda_reg):
    """
    Computes the Ridge regression objective function value.

    Parameters:
    - beta: Coefficient vector (numpy array of shape (d,))
    - X: Feature matrix (numpy array of shape (n_samples, n_features))
    - y: Target vector (numpy array of shape (n_samples,))
    - lambda_reg: Regularization parameter (float)

    Returns:
    - Objective function value (float)
    """
    n = X.shape[0]
    residual = X @ beta - y
    mse_term = (1 / (2 * n)) * np.dot(residual, residual)
    reg_term = (lambda_reg / 2) * np.dot(beta, beta)
    return mse_term + reg_term


In [14]:
#Fixed step size
from proxop.scalar import Square

n, d = X.shape

# Parameters
lambda_reg = 0.1  # Regularization parameter (λ)
step_size = 1 / L
max_iter = 1000
epsilon = 1e-6


# Initialize parameters
beta = np.ones(d)

# Initialize the Square proximal operator
square_prox = Square()

start_time = time.time()
# Proximal Gradient Descent loop
for i in range(max_iter):
    gradient = (1 / n) * X.T @ (X @ beta - y)  # Gradient of MSE
    beta_prev = beta.copy()
    beta_temp = beta - step_size * gradient
    # Apply proximal operator for L2 regularization (ridge penalty)
    beta = square_prox.prox(beta_temp, gamma=step_size * lambda_reg)
    objective_value = ridge_objective(beta, X, y, lambda_reg)
    print(f"Iter {i}: Obj = {objective_value:.6f}")

    # Check for convergence after minimum iterations
    if abs(ridge_objective(beta,X,y,lambda_reg) - ridge_objective(beta_prev,X,y,lambda_reg))/max(1,abs(ridge_objective(beta_prev,X,y,lambda_reg))) < epsilon:
        print(f"Converged in {i} iterations")
        break

end_time = time.time()

print(f"Obj = {ridge_objective(beta,X,y,lambda_reg):.6f}")
print(f"Running time= {end_time-start_time}")

Iter 0: Obj = 8440.417923
Iter 1: Obj = 5724.829627
Iter 2: Obj = 4267.495296
Iter 3: Obj = 3484.399518
Iter 4: Obj = 3063.143841
Iter 5: Obj = 2836.306536
Iter 6: Obj = 2714.034647
Iter 7: Obj = 2648.051802
Iter 8: Obj = 2612.396362
Iter 9: Obj = 2593.095745
Iter 10: Obj = 2582.624159
Iter 11: Obj = 2576.924924
Iter 12: Obj = 2573.809412
Iter 13: Obj = 2572.095605
Iter 14: Obj = 2571.144310
Iter 15: Obj = 2570.609321
Iter 16: Obj = 2570.302735
Iter 17: Obj = 2570.122297
Iter 18: Obj = 2570.012168
Iter 19: Obj = 2569.941717
Iter 20: Obj = 2569.894043
Iter 21: Obj = 2569.859747
Iter 22: Obj = 2569.833550
Iter 23: Obj = 2569.812447
Iter 24: Obj = 2569.794700
Iter 25: Obj = 2569.779281
Iter 26: Obj = 2569.765565
Iter 27: Obj = 2569.753161
Iter 28: Obj = 2569.741813
Iter 29: Obj = 2569.731350
Iter 30: Obj = 2569.721648
Iter 31: Obj = 2569.712616
Iter 32: Obj = 2569.704184
Iter 33: Obj = 2569.696295
Iter 34: Obj = 2569.688904
Iter 35: Obj = 2569.681970
Iter 36: Obj = 2569.675459
Iter 37: Ob

In [15]:
#Adaptive step size
from proxop.scalar import Square

n, d = X.shape

# Parameters
lambda_reg = 0.1
t_init = 1
max_iter = 1000
epsilon = 1e-6


# Initialize parameters
beta = np.ones(d)

# Initialize the Square proximal operator
square_prox = Square()
start_time = time.time()
# Proximal Gradient Descent loop
for i in range(max_iter):
    gradient = (1 / n) * X.T @ (X @ beta - y)  # Gradient of MSE
    # Use backtracking line search to find an appropriate step size
    eta = backtracking_line_search(X, y, beta, gradient, L,t_init)
    t_init=eta
    beta_prev = beta.copy()
    beta_temp = beta - eta * gradient
    beta = square_prox.prox(beta_temp, gamma=eta * lambda_reg)
    # Apply proximal operator for L2 regularization (ridge)
    objective_value = ridge_objective(beta, X, y, lambda_reg)
    print(f"Iter {i}: Obj = {objective_value:.6f}")

    # Check for convergence after minimum iterations
    if abs(ridge_objective(beta,X,y,lambda_reg) - ridge_objective(beta_prev,X,y,lambda_reg))/max(1,abs(ridge_objective(beta_prev,X,y,lambda_reg))) < epsilon:
        print(f"Converged in {i} iterations")
        break

end_time = time.time()

print(f"Obj = {ridge_objective(beta,X,y,lambda_reg):.6f}")
print(f"Running time= {end_time-start_time}")

Iter 0: Obj = 8873.456029
Iter 1: Obj = 6198.712663
Iter 2: Obj = 4663.028166
Iter 3: Obj = 3778.331432
Iter 4: Obj = 3268.196945
Iter 5: Obj = 2973.809240
Iter 6: Obj = 2803.793747
Iter 7: Obj = 2705.527289
Iter 8: Obj = 2648.679928
Iter 9: Obj = 2615.758856
Iter 10: Obj = 2596.668919
Iter 11: Obj = 2585.580742
Iter 12: Obj = 2579.126164
Iter 13: Obj = 2575.357809
Iter 14: Obj = 2573.148924
Iter 15: Obj = 2571.846988
Iter 16: Obj = 2571.073714
Iter 17: Obj = 2570.609510
Iter 18: Obj = 2570.326693
Iter 19: Obj = 2570.150871
Iter 20: Obj = 2570.038587
Iter 21: Obj = 2569.964372
Iter 22: Obj = 2569.913237
Iter 23: Obj = 2569.876314
Iter 24: Obj = 2569.848321
Iter 25: Obj = 2569.826085
Iter 26: Obj = 2569.807678
Iter 27: Obj = 2569.791916
Iter 28: Obj = 2569.778056
Iter 29: Obj = 2569.765623
Iter 30: Obj = 2569.754307
Iter 31: Obj = 2569.743900
Iter 32: Obj = 2569.734255
Iter 33: Obj = 2569.725267
Iter 34: Obj = 2569.716860
Iter 35: Obj = 2569.708971
Iter 36: Obj = 2569.701554
Iter 37: Ob

In [16]:
#Fixed step size
from proxop import L1Norm, Square

def elastic_net_objective(beta, X, y, lambda_reg, alpha):
  """
    Computes the Elastic Net objective function:
        (1/2n) * ||Xβ - y||^2 + λ * [α * ||β||_1 + (1 - α)/2 * ||β||_2^2]

    Parameters:
        beta : ndarray
            Coefficient vector.
        X : ndarray
            Feature matrix.
        y : ndarray
            Target vector.
        lambda_reg : float
            Regularization strength.
        alpha : float
            Elastic net mixing parameter (0 = Ridge, 1 = Lasso).

    Returns:
        float: The value of the Elastic Net objective function.
  """
  n = X.shape[0]
  residual = X @ beta - y
  mse_term = (1 / (2 * n)) * np.dot(residual, residual)
  l1_term = lambda_reg * alpha * np.sum(np.abs(beta))
  l2_term = (lambda_reg * (1 - alpha) / 2) * np.dot(beta, beta)
  return mse_term + l1_term + l2_term

# Sample data
n, d = X.shape

# Parameters
lambda_reg = 0.1  # Regularization strength
alpha = 0.027    # Mixing parameter between L1 and L2
step_size = 1 / L
max_iter = 1000
epsilon = 1e-6

# Initialize parameters
beta = np.ones(d)

# Initialize proximal operators
prox_l1 = L1Norm(lambda_reg * alpha)
prox_l2 = Square()
start_time = time.time()
# Proximal Gradient Descent loop
for i in range(max_iter):
    # Compute the gradient of the MSE term
    gradient = (1 / n) * X.T @ (X @ beta - y)
    beta_prev = beta.copy()

    # Gradient step
    beta_temp = beta - step_size * gradient

    # Apply the proximal operator for the squared L2 norm
    beta_temp = prox_l2.prox(beta_temp, gamma=step_size * lambda_reg * (1 - alpha))

    # Apply the proximal operator for the L1 norm
    beta = prox_l1.prox(beta_temp, gamma=step_size)

    # Compute the objective function value
    obj_value = elastic_net_objective(beta, X, y, lambda_reg, alpha)
    print(f"Iter {i}: Obj = {obj_value:.6f}")

    # Check for convergence
    if abs(elastic_net_objective(beta,X,y,lambda_reg, alpha) - elastic_net_objective(beta_prev,X,y,lambda_reg, alpha))/max(1,abs(elastic_net_objective(beta_prev,X,y,lambda_reg, alpha))) < epsilon:
        print(f"Converged in {i} iterations")
        break

end_time = time.time()
print(f"Obj={elastic_net_objective(beta,X,y,lambda_reg, alpha):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8463.444028
Iter 1: Obj = 5752.718214
Iter 2: Obj = 4292.656718
Iter 3: Obj = 3503.049589
Iter 4: Obj = 3073.726523
Iter 5: Obj = 2839.341682
Iter 6: Obj = 2710.775573
Iter 7: Obj = 2639.782685
Iter 8: Obj = 2600.297109
Iter 9: Obj = 2578.096871
Iter 10: Obj = 2565.458466
Iter 11: Obj = 2558.156050
Iter 12: Obj = 2553.863768
Iter 13: Obj = 2551.292301
Iter 14: Obj = 2549.720419
Iter 15: Obj = 2548.740076
Iter 16: Obj = 2548.117181
Iter 17: Obj = 2547.715191
Iter 18: Obj = 2547.452913
Iter 19: Obj = 2547.281022
Iter 20: Obj = 2547.168855
Iter 21: Obj = 2547.096896
Iter 22: Obj = 2547.052433
Iter 23: Obj = 2547.027008
Iter 24: Obj = 2547.014880
Iter 25: Obj = 2547.012087
Iter 26: Obj = 2547.015857
Iter 27: Obj = 2547.024229
Iter 28: Obj = 2547.035798
Iter 29: Obj = 2547.049558
Iter 30: Obj = 2547.064777
Iter 31: Obj = 2547.080926
Iter 32: Obj = 2547.097620
Iter 33: Obj = 2547.114577
Iter 34: Obj = 2547.131592
Iter 35: Obj = 2547.148518
Iter 36: Obj = 2547.165244
Iter 37: Ob

In [17]:
#Adaptive step size
from proxop import L1Norm, Square

def elastic_net_objective(beta, X, y, lambda_reg, alpha):
    """
    Computes the Elastic Net objective function value.
    """
    n = X.shape[0]
    residual = X @ beta - y
    mse_term = (1 / (2 * n)) * np.dot(residual, residual)
    l1_term = lambda_reg * alpha * np.sum(np.abs(beta))
    l2_term = (lambda_reg * (1 - alpha) / 2) * np.dot(beta, beta)
    return mse_term + l1_term + l2_term

# Sample data
n, d = X.shape

# Parameters
lambda_reg = 0.1  # Regularization strength
alpha = 0.027    # Mixing parameter between L1 and L2
t_init = 1
max_iter = 1000
epsilon = 1e-6

# Initialize parameters
beta = np.ones(d)

# Initialize proximal operators
prox_l1 = L1Norm(lambda_reg * alpha)
prox_l2 = Square()
start_time = time.time()
# Proximal Gradient Descent loop
for i in range(max_iter):
    # Compute the gradient of the MSE term
    gradient = (1 / n) * X.T @ (X @ beta - y)
    eta = backtracking_line_search(X, y, beta, gradient, L,t_init)
    t_init=eta
    beta_prev = beta.copy()

    # Gradient step
    beta_temp = beta - eta * gradient

    # Apply the proximal operator for the squared L2 norm
    beta_temp = prox_l2.prox(beta_temp, gamma=eta * lambda_reg * (1 - alpha))

    # Apply the proximal operator for the L1 norm
    beta = prox_l1.prox(beta_temp, gamma=eta)

    # Compute the objective function value
    obj_value = elastic_net_objective(beta, X, y, lambda_reg, alpha)
    print(f"Iter {i}: Obj = {obj_value:.6f}")

    # Check for convergence
    if abs(elastic_net_objective(beta,X,y,lambda_reg, alpha) - elastic_net_objective(beta_prev,X,y,lambda_reg, alpha))/max(1,abs(elastic_net_objective(beta_prev,X,y,lambda_reg, alpha))) < epsilon:
        print(f"Converged in {i} iterations")
        break
end_time = time.time()

print(f"Obj={elastic_net_objective(beta,X,y,lambda_reg, alpha):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8900.329010
Iter 1: Obj = 6227.687846
Iter 2: Obj = 4690.923629
Iter 3: Obj = 3801.259054
Iter 4: Obj = 3283.917476
Iter 5: Obj = 2982.166950
Iter 6: Obj = 2805.589520
Iter 7: Obj = 2701.844376
Iter 8: Obj = 2640.554870
Iter 9: Obj = 2604.138993
Iter 10: Obj = 2582.321197
Iter 11: Obj = 2569.124799
Iter 12: Obj = 2561.053901
Iter 13: Obj = 2556.054373
Iter 14: Obj = 2552.912952
Iter 15: Obj = 2550.908477
Iter 16: Obj = 2549.608939
Iter 17: Obj = 2548.753093
Iter 18: Obj = 2548.181171
Iter 19: Obj = 2547.794171
Iter 20: Obj = 2547.529820
Iter 21: Obj = 2547.348290
Iter 22: Obj = 2547.223654
Iter 23: Obj = 2547.138718
Iter 24: Obj = 2547.081867
Iter 25: Obj = 2547.045116
Iter 26: Obj = 2547.022880
Iter 27: Obj = 2547.011197
Iter 28: Obj = 2547.007217
Iter 29: Obj = 2547.008868
Converged in 29 iterations
Obj=2547.008868
Running time= 0.009501218795776367


In [18]:
#Fixed step size
from proxop import L1Norm, Square

def elastic_net_objective(beta, X, y, lambda_reg, alpha):
    """
    Computes the Elastic Net objective function value.
    """
    n = X.shape[0]
    residual = X @ beta - y
    mse_term = (1 / (2 * n)) * np.dot(residual, residual)
    l1_term = lambda_reg * alpha * np.sum(np.abs(beta))
    l2_term = (lambda_reg * (1 - alpha) / 2) * np.dot(beta, beta)
    return mse_term + l1_term + l2_term

# Sample data

n, d = X.shape

# Parameters
lambda_reg = 0.1  # Regularization strength
alpha = 0.027      # Mixing parameter between L1 and L2
step_size = 1 / L
max_iter = 1000
epsilon = 1e-6


# Initialize parameters
beta = np.ones(d)
theta = beta.copy()
t = 1

# Initialize proximal operators
prox_l1 = L1Norm(lambda_reg * alpha)
prox_l2 = Square()
start_time = time.time()
# FISTA loop
for i in range(max_iter):
    beta_prev = beta.copy()

    # Compute the gradient of the MSE term
    gradient = (1 / n) * X.T @ (X @ theta - y)

    # Gradient step
    beta_temp = theta - step_size * gradient

    # Apply the proximal operator for the squared L2 norm
    beta_temp = prox_l2.prox(beta_temp, gamma=step_size * lambda_reg * (1 - alpha))

    # Apply the proximal operator for the L1 norm
    beta_next = prox_l1.prox(beta_temp, gamma=step_size)

    # Update the momentum term
    t_next = (1 + np.sqrt(1 + 4 * t ** 2)) / 2
    theta = beta_next + ((t - 1) / t_next) * (beta_next - beta)


    # Compute the objective function value
    obj_value = elastic_net_objective(beta_next, X, y, lambda_reg, alpha)
    print(f"Iter {i}: Obj = {obj_value:.6f}")

    # Check for convergence after minimum iterations
    if abs(elastic_net_objective(beta_next,X,y,lambda_reg, alpha) - elastic_net_objective(beta,X,y,lambda_reg, alpha))/max(1,abs(elastic_net_objective(beta,X,y,lambda_reg, alpha))) < epsilon:
        print(f"Converged in {i} iterations")
        break
    # Update variables
    beta = beta_next
    t = t_next



end_time = time.time()
print(f"Obj={elastic_net_objective(beta,X,y,lambda_reg, alpha):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8463.444028
Iter 1: Obj = 5752.718214
Iter 2: Obj = 3960.875754
Iter 3: Obj = 3022.310708
Iter 4: Obj = 2648.769991
Iter 5: Obj = 2552.580542
Iter 6: Obj = 2548.139555
Iter 7: Obj = 2554.179981
Iter 8: Obj = 2552.773951
Iter 9: Obj = 2548.480998
Iter 10: Obj = 2546.405508
Iter 11: Obj = 2547.032077
Iter 12: Obj = 2548.499384
Iter 13: Obj = 2549.329800
Iter 14: Obj = 2549.252880
Iter 15: Obj = 2548.700204
Iter 16: Obj = 2548.132466
Iter 17: Obj = 2547.757056
Iter 18: Obj = 2547.579484
Iter 19: Obj = 2547.539748
Iter 20: Obj = 2547.586600
Iter 21: Obj = 2547.683768
Iter 22: Obj = 2547.796887
Iter 23: Obj = 2547.891971
Iter 24: Obj = 2547.944644
Iter 25: Obj = 2547.947435
Iter 26: Obj = 2547.908478
Iter 27: Obj = 2547.844121
Iter 28: Obj = 2547.771160
Iter 29: Obj = 2547.702205
Iter 30: Obj = 2547.644430
Iter 31: Obj = 2547.600305
Iter 32: Obj = 2547.569032
Iter 33: Obj = 2547.548025
Iter 34: Obj = 2547.534212
Iter 35: Obj = 2547.525007
Iter 36: Obj = 2547.518826
Iter 37: Ob

In [19]:
#Adaptive step size
from proxop import L1Norm, Square

def elastic_net_objective(beta, X, y, lambda_reg, alpha):
    """
    Computes the Elastic Net objective function value.
    """
    n = X.shape[0]
    residual = X @ beta - y
    mse_term = (1 / (2 * n)) * np.dot(residual, residual)
    l1_term = lambda_reg * alpha * np.sum(np.abs(beta))
    l2_term = (lambda_reg * (1 - alpha) / 2) * np.dot(beta, beta)
    return mse_term + l1_term + l2_term

# Sample data

n, d = X.shape

# Parameters
lambda_reg = 0.1  # Regularization strength
alpha = 0.027      # Mixing parameter between L1 and L2
t_init = 1
max_iter = 1000
epsilon = 1e-6


# Initialize parameters
beta = np.ones(d)
theta = beta.copy()
t = 1

# Initialize proximal operators
prox_l1 = L1Norm(lambda_reg * alpha)
prox_l2 = Square()
start_time = time.time()
# FISTA loop
for i in range(max_iter):
    beta_prev = beta.copy()

    # Compute the gradient of the MSE term
    gradient = (1 / n) * X.T @ (X @ theta - y)
    eta = backtracking_line_search(X, y, beta, gradient, L,t_init)
    t_init=eta
    # Gradient step
    beta_temp = theta - eta * gradient

    # Apply the proximal operator for the squared L2 norm
    beta_temp = prox_l2.prox(beta_temp, gamma=eta * lambda_reg * (1 - alpha))

    # Apply the proximal operator for the L1 norm
    beta_next = prox_l1.prox(beta_temp, gamma=eta)

    # Update the momentum term
    t_next = (1 + np.sqrt(1 + 4 * t ** 2)) / 2
    theta = beta_next + ((t - 1) / t_next) * (beta_next - beta)


    # Compute the objective function value
    obj_value = elastic_net_objective(beta_next, X, y, lambda_reg, alpha)
    print(f"Iter {i}: Obj = {obj_value:.6f}")

    # Check for convergence after minimum iterations
    if abs(elastic_net_objective(beta_next,X,y,lambda_reg, alpha) - elastic_net_objective(beta,X,y,lambda_reg, alpha))/max(1,abs(elastic_net_objective(beta,X,y,lambda_reg, alpha))) < epsilon:
        print(f"Converged in {i} iterations")
        break
    # Update variables
    beta = beta_next
    t = t_next


end_time = time.time()

print(f"Obj={elastic_net_objective(beta,X,y,lambda_reg, alpha):.6f}")
print(f"Running time= {end_time-start_time}")


Iter 0: Obj = 8900.329010
Iter 1: Obj = 6227.687846
Iter 2: Obj = 4331.983719
Iter 3: Obj = 3237.150036
Iter 4: Obj = 2735.897466
Iter 5: Obj = 2571.172670
Iter 6: Obj = 2545.999117
Iter 7: Obj = 2553.440199
Iter 8: Obj = 2556.277940
Iter 9: Obj = 2552.259223
Iter 10: Obj = 2547.786516
Iter 11: Obj = 2546.354045
Iter 12: Obj = 2547.404297
Iter 13: Obj = 2548.954874
Iter 14: Obj = 2549.710683
Iter 15: Obj = 2549.524742
Iter 16: Obj = 2548.865005
Iter 17: Obj = 2548.203640
Iter 18: Obj = 2547.757698
Iter 19: Obj = 2547.533937
Iter 20: Obj = 2547.464779
Iter 21: Obj = 2547.492715
Iter 22: Obj = 2547.582801
Iter 23: Obj = 2547.705466
Iter 24: Obj = 2547.827080
Iter 25: Obj = 2547.916147
Iter 26: Obj = 2547.954193
Iter 27: Obj = 2547.940313
Iter 28: Obj = 2547.887180
Iter 29: Obj = 2547.812973
Iter 30: Obj = 2547.734399
Iter 31: Obj = 2547.663175
Iter 32: Obj = 2547.605510
Iter 33: Obj = 2547.563117
Iter 34: Obj = 2547.534631
Iter 35: Obj = 2547.516967
Iter 36: Obj = 2547.506531
Iter 37: Ob